In [3]:
!pip install onnx onnxruntime

In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input1_name = "A"
input2_name = "B"
matmul_output_name = "Y"


# Create the ONNX model with Matmul operator
def create_matmul_model(dtype, output_rank):

    #Create "input-rank" inputs tensors
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, [None,None])
    input2 = onnx.helper.make_tensor_value_info(input2_name, dtype, [None,None])

    # Create output tensor (final result after matmul operation)
    matmul_output = onnx.helper.make_tensor_value_info(matmul_output_name, dtype, output_rank)

    # Define matmul node
    matmul_node = onnx.helper.make_node(
        "MatMul",
        inputs=[input1_name, input2_name],
        outputs=[matmul_output_name],
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [matmul_node],
        "MatMul",
        [input1, input2],
        [matmul_output],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "matmul.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("matmul.onnx")
    return session

def do_matmul(a, b, session):
    # Run inference
    output = session.run(None, {input1_name: a, input2_name: b})

    a_f = (np.array2string(a, separator=',', max_line_width=np.inf).replace('\n', ''))
    b_f = (np.array2string(b, separator=',', max_line_width=np.inf).replace('\n', ''))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"A={a_f}, B={b_f}")
    print(f"Result = {y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [2]:
# Case N1: 2 simple matrices multiplication
onnx_type = onnx.TensorProto.INT32
a = np.array([[1, 2], [3, 4]], dtype=np.int32)
b = np.array([[5, 6], [7, 8]], dtype=np.int32)
session = create_matmul_model(onnx_type, [2,2])
do_matmul(a, b, session)

A=[[1,2], [3,4]], B=[[5,6], [7,8]]
Result = [[19,22], [43,50]]


In [4]:
# Case N2: 2 simple matrices multiplication
onnx_type = onnx.TensorProto.INT32
a = np.array([[0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]], dtype=np.int32)
b = np.identity(5, dtype=np.int32)
session = create_matmul_model(onnx_type, [5,5])
do_matmul(a, b, session)

A=[[0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]], B=[[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]
Result = [[0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]]


In [ ]:
# Case N3: 2 simple matrices multiplication
onnx_type = onnx.TensorProto.INT32
a = np.array([[1,2], [3,4], [5,6]], dtype=np.int32)
b = np.array([[7,8,9], [10,11,12]], dtype=np.int32)
session = create_matmul_model(onnx_type, [3,3])
do_matmul(a, b, session)

A=[[1,2], [3,4], [5,6]], B=[[ 7, 8, 9], [10,11,12]]
Result = [[ 27, 30, 33], [ 61, 68, 75], [ 95,106,117]]


In [ ]:
#Case N4: 2 simple matrices multiplication
onnx_type = onnx.TensorProto.INT32
a = np.array([[1], [2], [3]], dtype=np.int32)
b = np.array([[4, 5, 6]], dtype=np.int32)
session = create_matmul_model(onnx_type, [3,3])
do_matmul(a, b, session)

(3, 1)
(1, 3)
A=[[1], [2], [3]], B=[[4,5,6]]
Result = [[ 4, 5, 6], [ 8,10,12], [12,15,18]]


In [10]:
#Case N5: 2 simple matrices multiplication
onnx_type = onnx.TensorProto.INT32
a = np.array([[4, 5, 6]], dtype=np.int32)
b = np.array([[1], [2], [3]], dtype=np.int32)
session = create_matmul_model(onnx_type, [1,1])
do_matmul(a, b, session)

A=[[4,5,6]], B=[[1], [2], [3]]
Result = [[32]]


## No nominal cases (nan and inf values)

In [ ]:
#Case N1: Infs and Nans
onnx_type = onnx.TensorProto.FLOAT
a = np.array([[np.inf, -np.inf, np.nan], [np.nan, np.inf, -np.inf]], dtype=np.float32)
b = np.array([[1, 2], [4, 5], [7, 8]], dtype=np.float32)
session = create_matmul_model(onnx_type, [2,2])
do_matmul(a, b, session)

A=[[ inf,-inf, nan], [ nan, inf,-inf]], B=[[1.00000000,2.00000000], [4.00000000,5.00000000], [7.00000000,8.00000000]]
Result = [[nan,nan], [nan,nan]]


In [ ]:
#Case N2: Infs and Nans
onnx_type = onnx.TensorProto.FLOAT
a = np.array([[np.inf, np.inf], [np.nan, np.inf]], dtype=np.float32)
b = np.array([[1, 2, 3, 4], [4, 5, 6, 7]], dtype=np.float32)
session = create_matmul_model(onnx_type, [2,4])
do_matmul(a, b, session)

A=[[inf,inf], [nan,inf]], B=[[1.00000000,2.00000000,3.00000000,4.00000000], [4.00000000,5.00000000,6.00000000,7.00000000]]
Result = [[inf,inf,inf,inf], [nan,nan,nan,nan]]


## Empty tensors

In [3]:
#Case N1: Empty tensors
onnx_type = onnx.TensorProto.INT32
a = np.array([], dtype=np.int32).reshape(1,0)
b = np.array([], dtype=np.int32).reshape(0,1)
session = create_matmul_model(onnx_type, [1,1])
do_matmul(a, b, session)

A=[], B=[]
Result = [[0]]


In [4]:
#Case N2: Empty tensors
onnx_type = onnx.TensorProto.INT32
a = np.array([], dtype=np.int32).reshape(0,1)
b = np.array([], dtype=np.int32).reshape(1,0)
session = create_matmul_model(onnx_type, [0,0])
do_matmul(a, b, session)

A=[], B=[]
Result = []
